In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from textblob import TextBlob
data = pd.read_csv('spam.csv')


def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree


data['Message'] = data['Message'].apply(lambda x: remove_punctuation(x))

data['Message'] = data['Message'].apply(lambda x: x.lower())

data['Message'] = data['Message'].apply(lambda x: re.split(r"\s+", x))

stopwords = nltk.corpus.stopwords.words('english')


def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output


data['Message'] = data['Message'].apply(lambda x: remove_stopwords(x))

lemm = nltk.stem.WordNetLemmatizer()

data['Message'] = data['Message'].apply(lambda x: [lemm.lemmatize(word) for word in x])
data

,Category,Message
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, dont, think, go, usf, life, around, though]"
...,...,...
5567,spam,"[2nd, time, tried, 2, contact, u, u, £750, pou..."
5568,ham,"[ü, b, going, esplanade, fr, home]"
5569,ham,"[pity, mood, soany, suggestion]"
5570,ham,"[guy, bitching, acted, like, id, interested, b..."


In [2]:
data['Message_String'] = data['Message'].apply(lambda x: ' '.join([item for item in x]))
data['Category'] = data['Category'].replace({'ham':0,'spam':1})
data.head()

,Category,Message,Message_String
0,0,"[go, jurong, point, crazy, available, bugis, n...",go jurong point crazy available bugis n great ...
1,0,"[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,1,"[free, entry, 2, wkly, comp, win, fa, cup, fin...",free entry 2 wkly comp win fa cup final tkts 2...
3,0,"[u, dun, say, early, hor, u, c, already, say]",u dun say early hor u c already say
4,0,"[nah, dont, think, go, usf, life, around, though]",nah dont think go usf life around though


In [4]:
lr = LogisticRegression(random_state=1)
y = data['Category']
x = data['Message_String']

In [6]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: Category, Length: 5572, dtype: int64

In [27]:
vectorizer1 = TfidfVectorizer()
vectorizer1.fit(x)
x_vect= vectorizer1.transform(x)
x_vect

<5572x8904 sparse matrix of type '<class 'numpy.float64'>'
	with 47221 stored elements in Compressed Sparse Row format>

In [28]:
df_vectorizer= pd.DataFrame(x_vect.todense(),columns=vectorizer1.get_feature_names())
df_vectorizer.head()

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,üll,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.model_selection import cross_val_score, cross_val_predict
vectorizer_score = cross_val_score(lr, x_vect, y, cv=3)
vectorizer_score.mean()

0.9764897181100265